프로덕션 환경으로 코드를 가져다 놓는 것은 애플리케이션을 개발에서 고객에게 가져가는 마지막 단계다. 프로덕션 환경으로 애플리케이션을 배포하ㅡㄴ 작업은 상당히 중요하지만 소프트웨어 아키텍트의 체크리스트에서 빠뜨릴 떄가 있다.

시스템이 개발 환경에서 동작하는 것처럼 프로덕션 환경에서도 잘 동작하리라 가정하지만 이는 매우 치명적인 실수다. 대부분의 프로덕션 시스템은 개발 환경의 구성과 매우 다르다. 이용할 수 있는 많은 최적화 기법과 디버깅, 개발 환경에서 당연하다고 여겨지는 것들이 프로덕션 환경에서는 사용할 수 없는 경우가 있다.

애플리케이션을 프로덕션 환경에 배포하는 일은 정밀한 과학이라기보다 예술에 가깝다. 시스템의 배포 복잡도는 시스템을 개발한 언어, 시스템의 런타임 이식성과 성능, 구성 파라미터 수, 시스템의 동종 환경이나 이종 환경으로의 배포 여부, 배포의 지리적 분포, 배포 자동화 도구, 다른 기타 요소에 영향을 받는다.

최근 몇 년 동안, 파이썬은 오픈소스 언어로써 자동화 수준이 높아졌으며 프로덕션 시스템으로의 패키지 배포를 지원하고 있다. 파이썬은 내장된 풍부한 가용성 및 서드 파티 지원 도구와 더불어, 프로덕션 배포와 배포 시스템을 최신으로 유지하는 고통과 번거로움을 감소시켰다.

배포 가능한 시스템 및 배포 용이성의 개념을 간단히 살펴보고, 파이썬을 사용해 프로덕션 시스템에서 실행되는 애플리케이션의 배포와 유지보수를 쉽게 하기 위해, 아키텍트가 레퍼토리에 추가할 수 있는 파이썬 애플리케이션의 배포 도구, 프로세스를 알아본다. 빈번하게 시스템을 중지하지 않아도 아키텍트가 프로덕션 시스템을 정상적이고 안전하게 유지하기 위해 적용할 수 있는 기법과 모범 사례도 살펴볼 것이다.

## 배포 용이성

소프트웨어 시스템의 배포 용이성은 소프트웨어를 개발 환경에서 프로덕션 환경으로 쉽게 가져갈 수 있는 정도다. 배포 용이성은 코드를 개발 환경에서 프로덕션 환경으로 배포하는 데 필요한 분리 단계 관점 즉, 공수 대비 인력-시간이나 복잡도 측면에서 측정할 수 있다.

개발 시스템이나 스테이징 시스템에서 잘 동작하는 코드가 프로덕션 시스템에서 유사한 방법으로 잘 동작할 것이라는 가정은 자주하는 실수다. 이런 실수가 일어나는 이유는 가끔 개발 시스템이 프로덕션 시스템과 매우 다른 요구사항을 갖고 있기 때문이다.

### 배포 용이성에 영향을 주는 요소들

* 최적화 및 디버깅: 개발 시스템에서는 코드의 최적화를 하지 않는 것이 일반적이다. 파이썬 같이 코드가 런타임에서 인터프리터 퍼리돼 실행될 때, 보통 프로그래머는 예외가 발생할 때 많은 양의 역추적 정보를 생성할 수 있도록 디버그 설정을 활성화한다. 대개 파이썬 인터프러티의 최적화는 비활성화돼 있다. 반면 프로덕션 시스템에서는 최적화가 활성화되고 디버깅은 비활성화된다. 이와 유사한 방식으로 코드를 동작시키려면 추가적인 구성이 필요하다. 물론 특정 상황에서는 프로그램을 최적화되지 않은 상태로 실행시키는 것보다 최적화 정도에 따라 다르게 동작시키는 것도 가능하다.

* 의존성과 버전: 일반적으로 개발 환경에는 개발자가 작업할 수 있는 다양한 애플리케이션을 실행하기 위한 풍부한 개발 라이브러리와 지원 라이브러리가 설치돼 있다. 대부분 개발자는 최신 코드로 작업하기 때문에 라이브러리 의존성을 갖고 있을 수 있다. 프로덕션 시스템은 미리 컴파일된 의존성 목록과 버전으로 주의 깊게 준비돼야 한다. 보통 프로덕션 시스템으로의 배포는 성숙하거나 안정된 버전만 지정한다. 따라서 개발자가 다운스트림에서 사용했던 기능을 불안정한 버전이나 버그 수정에 의존했다면, 프로덕션 환경에서 기능이 의도대로 동작하지 않는다는 사실을 너무 늦게 발견하기도 한다. 또 다른 문제를 문서화되지 않는 의존성이나 소스코드에서 컴파일해야 하는 의존성들인데 처음 배포할 때 문제를 일으키기도 한다.

* 리소스 구성과 액세스 권한: 개발 시스템과 프로덕션 시스템은 리소스 접근 수준과 권한, 세부 사항이 다르다. 개발 시스템은 로컬 데이터베이스를 가질 수 있지만 프로덕션 시스템은 애플리케이션과 데이터베이스 시스템을 별도로 호스팅하는 경향이 있다. 개발 시스템은 표준 구성 파일을 사용할 수 있다. 반면, 프로덕션 환경의 구성은 특정 스크립트를 사용해 호스트나 환경에 특화되게 생성할 수 있다. 비슷한 형태로 프로덕션 환경의 애플리케이션은 특정 사용자/그룹에서는 더 낮은 권한으로 실행될 수도 있지만 개발 환경에서는 프로그램을 root나 superuser로 실행한다. 사용자 권한과 구성의 불일치는 리소스 액세스에 영향을 줄 수 있으며 소프트웨어가 개발 환경에서 정상적으로 동작할 떄도 프로덕션 환경에서는 실패의 원인이 될 수 있다.

* 이기종 프로덕션 환경: 일반적으로 코드는 동질성을 갖는 개발 환경에서 개발된다. 그러나 코드는 프로덕션 환경의 이기종 시스템에 배포할 수 있어야 한다. 예를 들어 소프트웨어는 리눅스에서 개발될 수 있지만, 고객은 윈도우 관련 배포 요구사항을 갖고 있을 수 있다. 배포의 복잡성은 환경의 이질성 비례해 증가한다. 코드가 프로덕션 환경으로 가기 전, 잘 관리된 스테이징 환경과 테스팅 환경이 필요하다. 이기종 시스템은 각 대상 시스템의 아키텍처를 위한 별도의 의존성 목록을 관리해야 하기 때문에 의존성 관리를 더 어렵게 만든다.

* 보안: 개발 환경과 테스트 환경은 시간을 절약하고 테스팅을 위한 구성의 복잡도를 줄이기 위해 보안 측면에서 차이를 두는 것이 일반적이다. 예컨대 웹 애플리케이션에서 빠른 프로그래밍과 편리한 테스팅을 위해 특별한 개발 환경 플래그로 로그인이 필요한 경로를 비활성화할 수 있다. 유사하게 개발 환경에서 사용되는 데이터베이스 시스템, 웹 애플리케이션 로그인 시스템들은 일상적인 기억과 사용 편의를 위해 추측하기 쉬운 패스워드를 사용한다. 역할 기반으로 권한을 부여하는 것은 테스팅을 쉽게 만들기 위해 무시될 수 있다. 프로덕션 환경에서 보안은 중요하기 때문에 개발 환경과느 반대여야 한다.보안 측면에서 로그인이 필요한 경로는 개발 환경과 반대로 강력한 패스워드를 사용해야 한다. 그리고 역할 기반으로 권한을 부여해야 한다. 이것은 개발환경에서는 동작하지만 프로덕션 환경에서 실패하는 미묘한 버그의 원인이 된다.

## 소프트웨어 배포 아키텍처 계층

코드를 개발할 때 테스팅 및 프로덕션 호나경까지의 복잡성 방지를 위해, 프로덕션 환경으로 배포하기 전의 애플리케이션의 수명 주기 각 단계는 다중 계층 아키텍처를 사용한다.

* 개발/테스트/스테이지/프로덕션: 전통적인 4계층 아키텍처
    * 